# Llma-factory使用:

## 登陆HF账号

需要从HF拉取模型，所以需要先登陆HF账号。

In [1]:
# 安装或升级 huggingface_hub 库
!pip install --upgrade huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.23.5
    Uninstalling huggingface-hub-0.23.5:
      Successfully uninstalled huggingface-hub-0.23.5


In [2]:
# 登录 Hugging Face 账户(笔者创建的write权限token)
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
# 调用 whoami 函数获取当前用户的信息，确保自己登陆了HF
from huggingface_hub import whoami

user_info = whoami()
print(user_info)

{'type': 'user', 'id': '616980658b83fff6ec520d8d', 'name': 'peilong', 'fullname': 'chenpeilong', 'email': 'peilongchencc@163.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/47859410af6b3e6165c418162aab1f6f.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'hf_peilongchencc', 'role': 'write', 'createdAt': '2024-08-08T07:08:18.645Z'}}}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## 克隆Git仓库、安装依赖项

In [4]:
# 切换到 /content/ 目录
%cd /content/

# 删除 LLaMA-Factory 目录及其所有内容（如果存在）
%rm -rf LLaMA-Factory

# 从 GitHub 克隆 LLaMA-Factory 仓库
!git clone https://github.com/hiyouga/LLaMA-Factory.git

# 切换到刚刚克隆的 LLaMA-Factory 目录
%cd LLaMA-Factory

# 列出当前目录中的文件和文件夹
%ls

# 安装 LLaMA-Factory 项目及其依赖项（包括 torch 和 bitsandbytes）
!pip install -e .[torch,bitsandbytes]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 16003, done.
remote: Counting objects: 100% (6821/6821), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 16003 (delta 6538), reused 6531 (delta 6511), pack-reused 9182
Receiving objects: 100% (16003/16003), 222.68 MiB | 19.13 MiB/s, done.
Resolving deltas: 100% (11910/11910), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━

## 检查GPU资源:

In [5]:
import torch

try:
    # 断言检查是否有可用的 GPU
    assert torch.cuda.is_available() is True
except AssertionError:
    # 如果没有可用的 GPU，则抛出异常并打印提示信息
    print("需要 GPU 环境，申请教程：https://zhuanlan.zhihu.com/p/642542618")

## 更新自我认知数据集:

In [6]:
import json

%cd /content/LLaMA-Factory/  # 切换到指定目录

NAME = "Llama-Chinese"  # 定义常量NAME，表示模型名称
AUTHOR = "LLaMA Factory"  # 定义常量AUTHOR，表示作者名称

# 打开并读取data/identity.json文件，文件编码为utf-8
with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)  # 将文件内容加载为JSON对象

# 遍历JSON对象中的每个样本
for sample in dataset:
  # 替换样本中的占位符{{name}}和{{author}}为实际的NAME和AUTHOR值
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

# 将修改后的数据重新写回data/identity.json文件，确保文件格式美观且不转义非ASCII字符
with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

[Errno 2] No such file or directory: '/content/LLaMA-Factory/ # 切换到指定目录'
/content/LLaMA-Factory


## 设置Gradio的环境变量并启动LLaMA-Factory的可视化界面

设置 `GRADIO_SHARE=1` 意味着启用 Gradio 的共享功能，可以将你的 Gradio 应用分享到一个公共的URL，从而让其他人也可以访问和使用它。

In [ ]:
# 切换当前工作目录到LLaMA-Factory文件夹
%cd /content/LLaMA-Factory/

# 设置环境变量GRADIO_SHARE为1，启用Gradio共享功能，并启动llamafactory-cli的web界面
!GRADIO_SHARE=1 llamafactory-cli webui

/content/LLaMA-Factory
2024-08-08 08:17:48.120429: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-08 08:17:48.138959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 08:17:48.160354: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 08:17:48.166976: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-08 08:17:48.183006: I tensorflow/core/pl